In [24]:
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


#TRAINING DATA----------------------------------------------------------------------------------
df=pd.read_csv('training.csv')

#get rid of commas/punctuation
punctuation_signs = list("?:!.,;")
df['article_words_parse_1'] = df['article_words']

for punct_sign in punctuation_signs:
    df['article_words_parse_1'] = df['article_words_parse_1'].str.replace(punct_sign, ' ')
    
#all lower case
df['article_words_parse_2'] = df['article_words_parse_1'].str.lower()

#NO STEMMING NO LEMMISATION, NO STOP WORDS

#Category Numbers
category_codes = {
    'ARTS CULTURE ENTERTAINMENT': 0,
    'BIOGRAPHIES PERSONALITIES PEOPLE': 1,
    'DEFENCE': 2,
    'DOMESTIC MARKETS': 3,
    'FOREX MARKETS': 4,
    'HEALTH': 5,
    'IRRELEVANT': 6,
    'MONEY MARKETS': 7,
    'SCIENCE AND TECHNOLOGY': 8,
    'SHARE LISTINGS': 9,
    'SPORTS': 10
}

# Category mapping
df['Category_Code'] = df['topic']
df = df.replace({'Category_Code':category_codes})

text_train=df['article_words_parse_2']
y_train=df['Category_Code']

#TESTING DATA--------------------------------------------------------------------------------
df_t=pd.read_csv('test.csv')

#get rid of commas/punctuation
punctuation_signs = list("?:!.,;")
df_t['article_words_parse_1'] = df_t['article_words']

for punct_sign in punctuation_signs:
    df_t['article_words_parse_1'] = df_t['article_words_parse_1'].str.replace(punct_sign, ' ')
    
#all lower case
df_t['article_words_parse_2'] = df_t['article_words_parse_1'].str.lower()

#NO STEMMING NO LEMMISATION, NO STOP WORDS

#Category Numbers
category_codes = {
    'ARTS CULTURE ENTERTAINMENT': 0,
    'BIOGRAPHIES PERSONALITIES PEOPLE': 1,
    'DEFENCE': 2,
    'DOMESTIC MARKETS': 3,
    'FOREX MARKETS': 4,
    'HEALTH': 5,
    'IRRELEVANT': 6,
    'MONEY MARKETS': 7,
    'SCIENCE AND TECHNOLOGY': 8,
    'SHARE LISTINGS': 9,
    'SPORTS': 10
}

# Category mapping
df_t['Category_Code'] = df_t['topic']
df_t = df_t.replace({'Category_Code':category_codes})

text_test=df_t['article_words_parse_2']
y_test=df_t['Category_Code']

#count how many of each topic
print("Samples per class (training): {}".format(np.bincount(y_train)))
print("Samples per class (testing): {}".format(np.bincount(y_test)))

print("Number of documents in train data: {}".format(len(text_train)))
print("Number of documents in test data: {}".format(len(text_test)))

Samples per class (training): [ 117  167  258  133  845  183 4734 1673   70  218 1102]
Samples per class (testing): [  3  15  13   2  48  14 266  69   3   7  60]
Number of documents in train data: 9500
Number of documents in test data: 500


In [27]:
#textbook
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print("X_train:\n{}".format(repr(X_train)))
feature_names = vect.get_feature_names()

X_train:
<9500x35822 sparse matrix of type '<class 'numpy.int64'>'
	with 760419 stored elements in Compressed Sparse Row format>


In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
scores = cross_val_score(LogisticRegression(solver='lbfgs', multi_class='auto'), X_train, y_train, cv=5)
print("Mean cross-validation accuracy: {:.2f}".format(np.mean(scores)))

C:\Users\Stephen Sung\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Stephen Sung\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Stephen Sung\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Stephen Sung\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Mean cross-validation accuracy: 0.75


C:\Users\Stephen Sung\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [31]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)

C:\Users\Stephen Sung\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Stephen Sung\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Best cross-validation score: 0.76
Best parameters:  {'C': 0.01}


In [34]:
print("{:.2f}".format(grid.score(X_test, y_test)))

ValueError: could not convert string to float: 'world complet pharmaceut tianjin tianjin chin chin chin chin chin chin city coordin industr group group activit hoechst hoechst hoechst hoechst hoechst hoechst hoechst million million wide chemic capit invest invest invest invest invest dyestuff early offic offic year tijian strengen set northern found form azo joint joint joint percent percent side giant compan pigment pigment pigment pigment pigment pigment pigment schedul germ germ mln produc produc produc ton print market expect high fact export plast ventur ventur ventur ventur grad total ink thursday plant paint'